In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from worldcup.data import get_fixtures, get_results

In [99]:
cup_df = get_fixtures()

cup_df

,MatchNumber,RoundNumber,DateUtc,Location,HomeTeam,AwayTeam,Group,HomeTeamScore,AwayTeamScore
0,1,1,2022-11-20 16:00:00Z,Al Bayt Stadium,Qatar,Ecuador,Group A,0.0,2.0
1,3,1,2022-11-21 13:00:00Z,Khalifa International Stadium,England,Iran,Group B,6.0,2.0
2,2,1,2022-11-21 16:00:00Z,Al Thumama Stadium,Senegal,Netherlands,Group A,0.0,2.0
3,4,1,2022-11-21 19:00:00Z,Ahmad Bin Ali Stadium,USA,Wales,Group B,1.0,1.0
4,8,1,2022-11-22 10:00:00Z,Lusail Stadium,Argentina,Saudi Arabia,Group C,1.0,2.0
...,...,...,...,...,...,...,...,...,...
59,59,5,2022-12-10 19:00:00Z,TBA,To be announced,To be announced,None,NaN,NaN
60,61,6,2022-12-13 19:00:00Z,TBA,To be announced,To be announced,None,NaN,NaN
61,62,6,2022-12-14 19:00:00Z,TBA,To be announced,To be announced,None,NaN,NaN
62,63,7,2022-12-17 15:00:00Z,TBA,To be announced,To be announced,None,NaN,NaN


In [100]:
to_exclude = [
    '1A', '1C', '1D', '1B', '1E', '1G', '1F', '1H', 'To be announced', 
    '2B', '2D', '2C', '2A', '2F', '2H', '2E', '2G'
]

all_teams = [
    team #.replace('USA', 'United States').replace('Korea Republic', 'South Korea')
    for team in cup_df.AwayTeam.unique() if team not in to_exclude
]

all_teams


['Ecuador',
 'Iran',
 'Netherlands',
 'Wales',
 'Saudi Arabia',
 'Tunisia',
 'Poland',
 'Australia',
 'Croatia',
 'Japan',
 'Costa Rica',
 'Canada',
 'Cameroon',
 'Korea Republic',
 'Ghana',
 'Serbia',
 'Senegal',
 'USA',
 'Denmark',
 'Mexico',
 'Morocco',
 'Germany',
 'Switzerland',
 'Uruguay',
 'Qatar',
 'England',
 'France',
 'Argentina',
 'Belgium',
 'Spain',
 'Portugal',
 'Brazil']

In [35]:
# get 2 team goal diff time series

import pandas as pd

a, b = all_teams[0], all_teams[1]

a, b = 'Qatar', 'Ecuador'

a, b = 'USA', 'Wales'

In [91]:
df = get_results()

df[['home_team', 'away_team']]

df[['home_team', 'away_team']] = df[['home_team', 'away_team']].replace('United States', 'USA').replace('Korea Republic', 'South Korea')


# [team for team in df.home_team.unique() if 'U' in team]

In [62]:
past_df = df[df.home_team.isin(all_teams) & df.away_team.isin(all_teams)]

In [70]:
past_df['goal_diff_home'] = past_df.home_score - past_df.away_score

/var/folders/_n/rrz6mzy13l71h2mg878ykrv80000gn/T/ipykernel_47474/2193245238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_df['goal_diff_home'] = past_df.home_score - past_df.away_score


In [117]:
past_df = past_df.dropna(subset=['goal_diff_home']).reset_index().iloc[0:4224,:]


In [119]:
past_df['date'] = pd.to_datetime(past_df['date'])

In [114]:
# get fifa rankings

rank_df = pd.read_csv('../raw_data/fifa_ranking-2022-10-06.csv')

rank_df['country_full'] = rank_df['country_full'] \
    .replace('United States', 'USA') \
    .replace('IR Iran', 'Iran')

rank_df = rank_df[rank_df.country_full.isin(all_teams)]

len(sorted(rank_df.country_full.unique()))

# [team for team in rank_df.country_full.unique() if 'IR' in team]

32

In [115]:
rank_df['rank_date'] = pd.to_datetime(rank_df['rank_date'])

In [121]:
rank_df

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.00,0.00,0,UEFA,1992-12-31
35,113,Croatia,CRO,7.00,0.00,0,UEFA,1992-12-31
77,22,Cameroon,CMR,43.00,0.00,0,CAF,1992-12-31
79,24,USA,USA,43.00,0.00,0,CONCACAF,1992-12-31
80,25,Mexico,MEX,43.00,0.00,0,CONCACAF,1992-12-31
...,...,...,...,...,...,...,...,...
63857,21,Serbia,SRB,1563.62,1549.53,-4,UEFA,2022-10-06
63858,22,Morocco,MAR,1563.50,1558.35,-1,CAF,2022-10-06
63860,50,Qatar,QAT,1439.89,1441.97,2,AFC,2022-10-06
63861,51,Saudi Arabia,KSA,1437.78,1435.74,-2,AFC,2022-10-06


In [110]:
# for t in zip(sorted(all_teams), sorted(rank_df.country_full.unique())):
#     print(t)

('Argentina', 'Argentina')
('Australia', 'Australia')
('Belgium', 'Belgium')
('Brazil', 'Brazil')
('Cameroon', 'Cameroon')
('Canada', 'Canada')
('Costa Rica', 'Costa Rica')
('Croatia', 'Croatia')
('Denmark', 'Denmark')
('Ecuador', 'Ecuador')
('England', 'England')
('France', 'France')
('Germany', 'Germany')
('Ghana', 'Ghana')
('Iran', 'Japan')
('Japan', 'Korea Republic')
('Korea Republic', 'Mexico')
('Mexico', 'Morocco')
('Morocco', 'Netherlands')
('Netherlands', 'Poland')
('Poland', 'Portugal')
('Portugal', 'Qatar')
('Qatar', 'Saudi Arabia')
('Saudi Arabia', 'Senegal')
('Senegal', 'Serbia')
('Serbia', 'Spain')
('Spain', 'Switzerland')
('Switzerland', 'Tunisia')
('Tunisia', 'USA')
('USA', 'Uruguay')
('Uruguay', 'Wales')


In [ ]:
# create model

import statsmodels.api as sm

res = sm.tsa.arima.ARIMA(y, order=(1, 1, 1), trend='t').fit()
print(res.summary())

In [68]:
cup_df.iloc[0]['HomeTeam']
cup_df.iloc[0]['AwayTeam']

'Ecuador'

In [ ]:
# simulator
